# IBM Instance Setup

In [1]:
%cd ..

/Users/uribagi/Documents/GitHub/Latent-IQP


In [2]:
from dotenv import load_dotenv
import os
import jax
import jax.numpy as jnp
import iqpopt as iqp
from iqpopt.utils import initialize_from_data, local_gates
import iqpopt.gen_qml as genq
from iqpopt.gen_qml.utils import median_heuristic
from utils.nisq import aachen_connectivity, efficient_connectivity_gates
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session
import pennylane as qml
from datasets.bipartites import BipartiteGraphDataset
from datasets.er import ErdosRenyiGraphDataset
import numpy as np

key = jax.random.PRNGKey(42)

# Experiments Poster

In [3]:
bipartite_14 = jnp.array(BipartiteGraphDataset(nodes = 1, edge_prob=0.1).from_file('./datasets/raw_data/14N_Bipartite_Sparse.pkl').vectors.copy())
bipartite_18 = jnp.array(BipartiteGraphDataset(nodes = 1, edge_prob=0.1).from_file('./datasets/raw_data/18N_Bipartite_Sparse.pkl').vectors.copy())
erdos_renyi_18S = jnp.array(ErdosRenyiGraphDataset(nodes = 1, edge_prob=0.1).from_file('./datasets/raw_data/18N_ER_Sparse.pkl').vectors.copy())
erdos_renyi_14S = jnp.array(ErdosRenyiGraphDataset(nodes = 1, edge_prob=0.1).from_file('./datasets/raw_data/14N_ER_Sparse.pkl').vectors.copy())

[Dataset] Loaded 995 samples from ./datasets/raw_data/14N_Bipartite_Sparse.pkl
  Created: 2025-05-30T13:16:01.302922
  Unique graphs: 995
  Version: 1.0
[Dataset] Loaded 992 samples from ./datasets/raw_data/18N_Bipartite_Sparse.pkl
  Created: 2025-05-30T13:16:36.761074
  Unique graphs: 992
  Version: 1.0
[Dataset] Loaded 1000 samples from ./datasets/raw_data/18N_ER_Sparse.pkl
  Created: 2025-05-30T13:09:53.812165
  Unique graphs: 1000
  Version: 1.3
  Type: erdos_renyi
[Dataset] Loaded 1000 samples from ./datasets/raw_data/14N_ER_Sparse.pkl
  Created: 2025-05-30T13:09:28.785584
  Unique graphs: 1000
  Version: 1.3
  Type: erdos_renyi


In [4]:
def train_on_dataset(dataset, node_count):
    grid_conn= aachen_connectivity()
    num_qubits = node_count * (node_count - 1) // 2
    gates = efficient_connectivity_gates(grid_conn, num_qubits, 1)
    
    circuit = iqp.IqpSimulator(num_qubits, gates, device="lightning.qubit")
    
    initial_params = initialize_from_data(gates, dataset)
    loss = iqp.gen_qml.mmd_loss_iqp
    learning_rate = 0.003
    sigma = median_heuristic(dataset)
    
    loss_kwarg = {
        "params": initial_params,
        "iqp_circuit": circuit,
        "ground_truth": dataset,
        "sigma": [sigma],
        "n_ops": 2000,
        "n_samples": 2000,
        "key": jax.random.PRNGKey(42),
    }
    
    trainer = iqp.Trainer("Adam", loss, stepsize=learning_rate)
    trainer.train(n_iters= 2000,loss_kwargs=loss_kwarg, turbo=1)
    
    return trainer.final_params

In [5]:
params_b14 = train_on_dataset(bipartite_14, 14)
params_b18 = train_on_dataset(bipartite_18, 18)
params_er18s = train_on_dataset(erdos_renyi_18S, 18)
params_er14s = train_on_dataset(erdos_renyi_14S, 14)

Training Progress: 100%|██████████| 2000/2000 [00:57<00:00, 34.66it/s, loss=-0.000217, elapsed time=0.03, total time=58.4]


Training has not converged after 2000 steps


Training Progress: 100%|██████████| 2000/2000 [01:17<00:00, 25.77it/s, loss=-0.000247, elapsed time=0.04, total time=78.1]


Training has not converged after 2000 steps


Training Progress: 100%|██████████| 2000/2000 [01:14<00:00, 26.70it/s, loss=-0.000257, elapsed time=0.04, total time=75.1]


Training has not converged after 2000 steps


Training Progress: 100%|██████████| 2000/2000 [00:57<00:00, 35.08it/s, loss=-0.000301, elapsed time=0.03, total time=57.2]

Training has not converged after 2000 steps


# Hardware test

In [6]:
load_dotenv('.env')

ibm_token = os.getenv('IBM_TOKEN')
instance = os.getenv("INSTANCE")
setup = True 

if setup:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=ibm_token, overwrite=True)

service = QiskitRuntimeService(channel="ibm_cloud", token = ibm_token, instance=instance)
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=156)
print(backend)

<IBMBackend('ibm_aachen')>


In [7]:
dev = qml.device('qiskit.remote', 
                    wires=backend.num_qubits, 
                    backend=backend, 
                    shots=1024,
                    optimization_level=3)

In [8]:
grid_conn = aachen_connectivity()

gates_18 = efficient_connectivity_gates(grid_conn, 153, 1)
gates_14 = efficient_connectivity_gates(grid_conn, 91, 1)

In [9]:
IQP_18 = iqp.IqpSimulator(153, gates_18, device="lightning.qubit")
IQP_14 = iqp.IqpSimulator(91, gates_14, device="lightning.qubit")

In [10]:
@qml.qnode(dev)
def sample_18(trained_params):
    IQP_18.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(153))

@qml.qnode(dev)
def sample_14(trained_params):
    IQP_14.iqp_circuit(np.asarray(trained_params))
    return qml.sample(wires = range(91))

In [11]:
with Session(backend=backend, max_time="2h") as session:
    er14_results = sample_14(params_er14s)
    er18_results = sample_18(params_er18s)
    b14_results = sample_14(params_b14)
    b18_results = sample_18(params_b18)

In [12]:
er14_results

Array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 1, 0, 0]], dtype=int64)

# Analysis

In [13]:
from utils import metrics
from datasets import utils

In [27]:
print("Number of samples for 14N ER:", len(er14_results))
print("Number of samples for 18N ER:", len(er18_results))
print("Number of samples for 14N Bipartite:", len(b14_results))
print("Number of samples for 18N Bipartite:", len(b18_results))


Number of samples for 14N ER: 1024
Number of samples for 18N ER: 1024
Number of samples for 14N Bipartite: 1024
Number of samples for 18N Bipartite: 1024


In [28]:
# Save the results
np.save('./results/14N_ER_samples.npy', er14_results)
np.save('./results/18N_ER_samples.npy', er18_results)
np.save('./results/14N_Bipartite_samples.npy', b14_results)
np.save('./results/18N_Bipartite_samples.npy', b18_results)

In [29]:
# Save the trained parameters
np.save('./results/params_14N_ER.npy', params_er14s)
np.save('./results/params_18N_ER.npy', params_er18s)
np.save('./results/params_14N_Bipartite.npy', params_b14)
np.save('./results/params_18N_Bipartite.npy', params_b18)

# ER

In [15]:
graph_er14 = [utils.vec_to_graph(vec, 14) for vec in er14_results]
graph_er18 = [utils.vec_to_graph(vec, 18) for vec in er18_results]

ground_truth_er14 = [utils.vec_to_graph(vec, 14) for vec in erdos_renyi_14S]
ground_truth_er18 = [utils.vec_to_graph(vec, 18) for vec in erdos_renyi_18S]

In [16]:
edge_prob_er14 = [np.sum(vec)/(14 * (14 - 1) // 2) for vec in er14_results]
edge_prob_er18 = [np.sum(vec)/(18 * (18 - 1) // 2) for vec in er18_results]
print("Average edge probability for 14N ER samples:", np.mean(edge_prob_er14))
print("Average edge probability for 18N ER samples:", np.mean(edge_prob_er18))
print("Average edge probability for 14N ER ground truth:", np.mean([np.sum(vec)/(14 * (14 - 1) // 2) for vec in erdos_renyi_14S]))
print("Average edge probability for 18N ER ground truth:", np.mean([np.sum(vec)/(18 * (18 - 1) // 2) for vec in erdos_renyi_18S]))

Average edge probability for 14N ER samples: 0.19534469436813187
Average edge probability for 18N ER samples: 0.2003421160130719
Average edge probability for 14N ER ground truth: 0.15745054945054943
Average edge probability for 18N ER ground truth: 0.1496797385620915


In [30]:
# Number of correct samples (which have edge prob between 0.05 and 0.2)
correct_samples_er14 = [elem for elem in edge_prob_er14 if 0.05 <= elem <= 0.2]
correct_samples_er18 = [elem for elem in edge_prob_er18 if 0.05 <= elem <= 0.2]
print("Number of correct samples for 14N ER:", 100 * len(correct_samples_er14) / 1024)
print("Number of correct samples for 18N ER:", 100* len(correct_samples_er18) /1024)
# Calculate the average edge probability of the correct samples

Number of correct samples for 14N ER: 58.203125
Number of correct samples for 18N ER: 50.87890625


In [18]:
650 / 1024

0.634765625

In [ ]:
metrics.analyze_model_vs_dataset(ground_truth_er14, graph_er14);


=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 1000
   Generated graphs: 1024
   GT unique-adj  : 1000
   Gen unique-adj : 1024
   GT unique-iso  : 999
   Gen unique-iso : 1024

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              14.33    17.78     3.45      1.24
   Avg density             0.16     0.20     0.04      1.24
   Conn. frac              0.22     0.40     0.18      1.83
   #Comp                   3.25     2.00    -1.25      0.62
   Bip frac                0.28     0.07    -0.21      0.25
   Bip density             0.20     0.26     0.06      1.30

3) Memorization & coverage
   Precision (Gen→GT):  0.20%
   Recall    (GT←Gen):  0.20%
   Novel gen        : 1022/1024 (99.80%)
   GT never seen    : 998/1000 (99.80%)
   Matches per GT   : avg 1.00, med 1.00, min 1, max 1

4) Degree distribution divergences
   Avg paired KL(deg): 2.9494
   KL(GT→Gen)        : 0.0661
   KL(Gen

{'n_truth': 1000,
 'n_gen': 1024,
 'unique_adj_gt': 1000,
 'unique_adj_gen': 1024,
 'unique_iso_gt': 999,
 'unique_iso_gen': 1024,
 'gt_struct': {'n_graphs': 1000,
  'avg_nodes': 14.0,
  'std_nodes': 0.0,
  'avg_edges': 14.328,
  'std_edges': 4.773721399495367,
  'avg_density': 0.15745054945054943,
  'std_density': 0.0524584769175315,
  'fraction_connected': 0.218,
  'avg_components': 3.251,
  'fraction_bipartite': 0.279,
  'avg_bipartite_density': 0.20373189136629993,
  'std_bipartite_density': 0.04740261593677494},
 'gen_struct': {'n_graphs': 1024,
  'avg_nodes': 14.0,
  'std_nodes': 0.0,
  'avg_edges': 17.7763671875,
  'std_edges': 3.8058071486444685,
  'avg_density': 0.19534469436813187,
  'std_density': 0.041822056578510645,
  'fraction_connected': 0.3994140625,
  'avg_components': 2.0,
  'fraction_bipartite': 0.068359375,
  'avg_bipartite_density': 0.26476959831551666,
  'std_bipartite_density': 0.04678319889198143},
 'delta_avg_edges': 3.4483671875000006,
 'ratio_avg_edges': 1.2

In [24]:
metrics.analyze_model_vs_dataset(ground_truth_er18, graph_er18);


=== Model vs Dataset Analysis ===

1) Basic sizes & uniqueness
   GT graphs       : 1000
   Generated graphs: 1024
   GT unique-adj  : 1000
   Gen unique-adj : 1024
   GT unique-iso  : 1000
   Gen unique-iso : 1024

2) Structural statistics (GT vs Gen)
   Metric               GT       Gen       Δ        Ratio
   Avg edges              22.90    30.65     7.75      1.34
   Avg density             0.15     0.20     0.05      1.34
   Conn. frac              0.36     0.64     0.28      1.79
   #Comp                   3.28     1.46    -1.82      0.44
   Bip frac                0.18     0.00    -0.18      0.01
   Bip density             0.15     0.25     0.10      1.65

3) Memorization & coverage
   Precision (Gen→GT):  0.00%
   Recall    (GT←Gen):  0.00%
   Novel gen        : 1024/1024 (100.00%)
   GT never seen    : 1000/1000 (100.00%)
   Matches per GT   : avg 0.00, med 0.00, min 0, max 0

4) Degree distribution divergences
   Avg paired KL(deg): 3.5665
   KL(GT→Gen)        : 0.1628
   KL

## Bipartites

In [20]:
graph_b14 = [utils.vec_to_graph(vec, 14) for vec in b14_results]
graph_b18 = [utils.vec_to_graph(vec, 18) for vec in b18_results]

ground_truth_b14 = [utils.vec_to_graph(vec, 14) for vec in bipartite_14]
ground_truth_b18 = [utils.vec_to_graph(vec, 18) for vec in bipartite_18]

In [21]:
# Measure the number of correct samples (bipartites)
prop_b14 = metrics.bipartite_proportion(b14_results, 14)
prop_b18 = metrics.bipartite_proportion(b18_results, 18)

In [22]:
# Memorized samples
mem_b14 =  metrics.memorized_proportion(ground_truth_b14, graph_b14)
mem_b18 =  metrics.memorized_proportion(ground_truth_b18, graph_b18)

=== Memorization Report ===
Ground‐truth graphs:       995
Generated graphs:          1024
Generated ⟶ GT matches:    93/1024 ( 9.08%)
GT ⟵ Generated coverage:   84/995 ( 8.44%)
=== Memorization Report ===
Ground‐truth graphs:       992
Generated graphs:          1024
Generated ⟶ GT matches:    2/1024 ( 0.20%)
GT ⟵ Generated coverage:   2/992 ( 0.20%)


In [23]:
# Measure the number of unique samples (bipartites)
unique_b14 = metrics.sample_diversity(graph_b14)
unique_b18 = metrics.sample_diversity(graph_b18)

print(f"Proportion of bipartite samples for 14N Bipartite: {prop_b14}")
print(f"Proportion of bipartite samples for 18N Bipartite: {prop_b18}")
print(f"Memorized proportion for 14N Bipartite: {mem_b14}")
print(f"Memorized proportion for 18N Bipartite: {mem_b18}")
print(f"Unique samples for 14N Bipartite: {unique_b14}")
print(f"Unique samples for 18N Bipartite: {unique_b18}")

Proportion of bipartite samples for 14N Bipartite: 0.3251953125
Proportion of bipartite samples for 18N Bipartite: 0.0947265625
Memorized proportion for 14N Bipartite: {'n_truth': 995, 'n_gen': 1024, 'gen_mem_count': 93, 'gen_mem_fraction': 0.0908203125, 'truth_cov_count': 84, 'truth_cov_fraction': 0.08442211055276382}
Memorized proportion for 18N Bipartite: {'n_truth': 992, 'n_gen': 1024, 'gen_mem_count': 2, 'gen_mem_fraction': 0.001953125, 'truth_cov_count': 2, 'truth_cov_fraction': 0.0020161290322580645}
Unique samples for 14N Bipartite: (1015, 0.9912109375)
Unique samples for 18N Bipartite: (1024, 1.0)
